In [ ]:

데이터 불러오기부터
가. 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 나. 데이터 불러오기
df_feature = pd.read_csv("onenavi_train_feature.csv",sep="|")
df_target = pd.read_csv("onenavi_train_target.csv",sep="|")

In [ ]:
나. Train/Test Data Split

In [ ]:
from sklearn.model_selection import train_test_split

# train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_feature, df_target, test_size=0.20, random_state=42)

In [ ]:

모델 최적화 : Hyperparameter Tuning
Hyperparmeter는 머신러닝에서 모델을 학습하기 전에 설정해주는 옵션 값이라고 할 수 있습니다.
설정을 머신러닝이 스스로 찾을 수 있다면 참 좋을텐데
불가능하기 때문에 우리가 직접 찾아서 설정 해주어야 합니다.
모델을 최적화하는 전략은 크게 Panda전략과 Caviar전략으로 나누어볼 수 있는데요.
간단하게 두 전략의 개념을 짚어보고 실습하는 시간을 가지겠습니다.
지금부터 모델 최적화를 시작합니다.
1. Panda전략 : Babysitting one model
크게 하나의 모델을 정하고 성능을 평가하면서 점진적으로 모델을 '돌보기'하는 전략
한 번에 한마리씩 아기를 가지는 팬더는 아기 팬더가 살아남을 수 있도록 지극정성으로 케어를 하는데, 여기서도 동일하게 하나의 모델을 잘 케어해서 최적화하는 개념을 이야기한다.Image
[Panda 전략 Tip : XGBoost 기준]

모델 성능을 높이고 싶다면!
n-estimators는 높게, eta는 낮게
튜닝 예시
n-estimators와 learning_rate를 먼저 지정(n-estimators를 100으로 고정하고 최적 learning_rate 찾기)
나머지 hyperparameter 튜닝
n-estimators와 learning_rate 조정(둘은 반비례, 꼭 법칙은 아님!)
(예시) n-estimators를 2배 늘리면 learning_rate는 1/2로, n-estimators를 10배 늘리면 learning_rate는 1/10

In [ ]:
# 1단계 n-estimators와 learning_rate(eta)를 먼저 지정 : eta 0.1 => R-squared Score on Test set : 0.71631
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

model=xgb(n_estimators=100, eta=0.1)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# 1단계 n-estimators와 learning_rate(eta)를 먼저 지정 : eta 0.2 => R-squared Score on Test set : 0.71680
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

model=xgb(n_estimators=100, eta=0.2)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# 1단계 n-estimators와 learning_rate(eta)를 먼저 지정 : eta 0.3 => R-squared Score on Test set : 0.70988
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

model=xgb(n_estimators=100, eta=0.3)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# 2단계 나머지 hyperparameter 튜닝 : 0.71724
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

model=xgb(n_estimators=100, eta=0.2, 
          max_depth=5, subsample= 0.8, colsample_bytree=0.5, reg_alpha=3, gamma=5)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# 3단계 n-estimators와 learning_rate 조정 : 0.71890
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

model=xgb(n_estimators=200, eta=0.1,
          max_depth=5, subsample= 0.8, colsample_bytree=0.5, reg_alpha=3, gamma=5)
model.fit(train_x, train_y)

pred_y = model.predict(test_x)
print("RMSE on Test set : {0:.5f}".format(mean_squared_error(test_y,pred_y)**0.5))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(test_y,pred_y)))

In [ ]:
# 모델을 저장합니다.
import pickle
import joblib


joblib.dump(model, '4_model.pkl') 

In [ ]:

2. Caviar전략 : Training many models in parallel
여러개 모델을 동시에 돌려서 가장 좋은 모델을 선택하는 전략
한 철에 1억개의 알을 품는 물고기가 있다고 한다. 물고기가 번식하는 과정은 하나에 많은 집중을 쏟기보다 하나 또는 그 이상이 더 잘 살아남기를 그지 지켜보는데, 여기에서 착안하여 여러 모델을 시도해서 최적의 모델을 찾는 전략을 Caviar전략이라고 한다. image.png
Caviar 전략에서는 Tip이라고 정리하기보다 Gridsearch라는 유용한 라이브러리를 소개하고자 한다.
Gridsearch는 여러 모델을 코드 몇 줄로 시도하고 최적 파라미터를 도출하기에 편하다.
다만, 시간이 많이 걸린다.

In [ ]:
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import time

params = { 'n_estimators' : [50, 100, 200],
           'learning_rate' : [0, 0.01],
           'max_depth' : [0, 3],
            }

xgb_model = xgb(random_state = 0, n_jobs = 1)
grid_cv = GridSearchCV(xgb_model, param_grid = params, cv = 3, n_jobs = 1)
start_time = time.process_time()
grid_cv.fit(train_x, train_y)
end_time = time.process_time()

print('----  {0:.5f}sec, training complete  ----'.format(end_time-start_time))
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
3. 모델 평가

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as grb
from xgboost import XGBRegressor as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, r2_score

import pickle
import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
df_evaluation = pd.read_csv("onenavi_evaluation_new.csv",sep="|")
df_evaluation_feature = pd.read_csv("onenavi_eval_feature.csv",sep="|")

In [ ]:
model_rslt = []
for i in range(5):
    model_rslt.append(joblib.load("{}_model.pkl".format(i)))
model_rslt.append(keras.models.load_model("DeeplearningModel.h5"))
model_rslt

In [ ]:
e1_list = ['ETA1', 'ETA2', 'ETA3', 'ETA4', 'ETA5', 'ETA6']
e2_list = ['ETAA1', 'ETAA2', 'ETAA3', 'ETAA4', 'ETAA5', 'ETAA6']

for e1, e2, model in zip(e1_list, e2_list, model_rslt):
    df_evaluation[e1] = model.predict(df_evaluation_feature)
    df_evaluation.loc[(df_evaluation[e1] < 0), e1] = 0
    etaa = (1-(abs(df_evaluation['ET']-df_evaluation[e1])/df_evaluation['ET']))*100.0
    df_evaluation[e2] = etaa
    df_evaluation.loc[(df_evaluation[e2] < 0), e2] = 0

# mean, min, max, std
etaa = ['ETAA', 'ETAA1', 'ETAA2', 'ETAA3', 'ETAA4', 'ETAA5', 'ETAA6']
alg = ['DATA', 'ML-LG', 'ML-RFR', 'ML-GBR', 'XBR', 'XBR2', 'Deep']

print('+-------------------------------------------------------+')
print('|   ALG    | Mean(%) |    STD    |  MIN(%)  |  MAX(%)   |')
print('+----------+---------+-----------+----------+-----------+')
for i, e in zip(range(len(alg)), etaa):
    eMean = df_evaluation[e].mean()
    eStd = df_evaluation[e].std()
    eMin = df_evaluation[e].min()
    eMax = df_evaluation[e].max()
    print('|  {:6s}  |   {:3.1f}  |   {:05.1f}   |   {:4.1f}   |  {:7.1f}  |'.format(alg[i], eMean, eStd, eMin, eMax))
print('+----------+---------+-----------+----------+-----------+\n\n')

In [ ]:
좀 더 상세한 분석을 위해 거리 구간별로 결과를 확인해 보겠습니다.

In [ ]:
etaa = ['ETAA', 'ETAA1', 'ETAA2', 'ETAA3', 'ETAA4', 'ETAA5', 'ETAA6']
alg = ['DATA', 'ML-LG', 'ML-RFR', 'ML-GBR', 'XBR', 'XBR2', 'Deep']

for length in range(4):
    if length == 0:
        print(' 1,000 <= A_DISTANCE < 5,000m')
    elif length == 1:
        print(' 5,000 <= A_DISTANCE < 10,000m')
    elif length == 2:
        print(' 10,000 <= A_DISTANCE < 15,000m')
    else:
         print('All A_DISTANCE')
    print('+-------------------------------------------------------+')
    print('|   ALG    | Mean(%) |    STD    |  MIN(%)  |  MAX(%)   |')
    print('+----------+---------+-----------+----------+-----------+')
    for i, e in zip(range(len(alg)), etaa):
        if length == 0:
            eMean = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].mean()
            eStd = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].std()
            eMin = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].min()
            eMax = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=1000) & (df_evaluation['A_DISTANCE']<5000), e].max()
        elif length == 1:
            eMean = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].mean()
            eStd = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].std()
            eMin = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].min()
            eMax = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=5000) & (df_evaluation['A_DISTANCE']<10000), e].max()
        elif length == 2:
            eMean = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].mean()
            eStd = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].std()
            eMin = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].min()
            eMax = df_evaluation.loc[(df_evaluation['A_DISTANCE']>=10000) & (df_evaluation['A_DISTANCE']<15000), e].max()
        else:
            eMean = df_evaluation[e].mean()
            eStd = df_evaluation[e].std()
            eMin = df_evaluation[e].min()
            eMax = df_evaluation[e].max()
        print('|  {:6s}  |   {:3.1f}  |   {:05.1f}   |   {:4.1f}   |  {:7.1f}  |'.format(alg[i], eMean, eStd, eMin, eMax))
    print('+----------+---------+-----------+----------+-----------+\n\n')

In [ ]:
모델의 R-squared Score, RMSE가 좋았다고해서 실제 평가데이터에 적용했을 때,
비례해서 모델의 정확도 혹은 결과가 좋아지지 않는다는 사실을 알 수 있습니다.
물론 두 수치가 좋아지면 모델의 정확도 혹은 결과가 좋아지는 경향은 있으나,
정확하게 비례하지는 않습니다.
그래서 다각도로 보면서 모델을 만들어줘야합니다.
(과적합을 방지하고, 변수의 영향을 조정하는 등)